* Keras의 embeding layer와 사전 훈련된 워드임베딩(pre-trained word embedding)을 가져와서 사용하는 것을 비교
* 자연어 처리시 갖고 있는 훈련데이터의 단어들을 임베딩 층을 구현하여 임베딩 벡터로 학습하는 경우가 있음
* Keras에서는 이를 Embedding() 메소드를 사용하여 구현

---
* 위키피디아 등과 같이 방대한 코퍼스를 갖고 Word2Vec, FastText, GloVe 등을 통해 미리 훈련된 임베딩 벡터를 불러오는 방법을 사용하는 경우가 있음

### 1. Keras의 Embedding layer

#### 1) 임베딩 층은 룩업 테이블이다.

* 임베딩 층의 입력으로 사용하기 위해 입력 시퀀스의 각 단어들은 모두 정수 인코딩이 되어있어야 함
    * 특정단어 --> 단어에 부여된 고유 정수값 -> 임베딩 층 통과 -> dense vector
* 임베딩 층은 입력 정수에 대해 dense vector로 맵핑하고 이 dense vector는 인공신경망의 학습과정에서 가중치가 학습되는 것과 같은 방식으로 학습됨
* 학습과정에서 단어는 모델이 풀고자 하는 작업에 맞는 값으로 업데이트되는데, 이 dense vector를 임베딩 벡터라고 부름


---
* 정수를 dense vector 또는 embedding vector로 맵핑한다는 것은....
* **특정 단어와 맵핑되는 정수를 인덱스로 가지는 테이블로부터 임베딩 벡터값을 가져오는 룩업 테이블**이라고 볼 수 있음
* 그리고 이 테이블은 **단어 집합의 크기만큼 행**을 가지므로 모든 단어는 고유한 임베딩 벡터를 가짐
* 케라스는 단어를 정수 인덱스로 바꾸고 원-핫 벡터로 변환 후 임베딩 층의 입력으로 사용하는 것이 아니라, **단어를 정수 인코딩까지만 진행 후 임베딩 층의 입력으로 사용**하여 룩업 테이블 결과인 임베딩 벡터를 리턴함

##### 임베딩 층 구현 코드


* vocab_size = 텍스트 데이터의 전체 단어 집합의 크기. 즉 코퍼스의 크기
* output_dim = 워드 임베딩 후의 임베딩 벡터의 차원
* input_length = 입력 시퀀스의 길이. 만약 갖고있는 각 샘플의 길이가 500개라면 이 값은 500임.

----
* Embedding()은 (number of samples, input_length)인 2D 정수 텐서를 입력받고, 
* 이때 각 샘플은 정수 인코딩이 된 결과로 정수 시퀀스가 됨
* Embedding()은 워드 임베딩 작업을 수행하고 (number of samples, input_length, embedding word dimentionality) 3D 실수 텐서를 리턴


##### embedding layer 사용하기
* 문장의 긍, 부정을 판단하는 감성 분류 모델을 만들기. 문장과 레이블 데이터를 만들고 긍정인 문장은 레이블 1, 부정인 문장은 레이블이 0

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']

In [3]:
y_train = [1, 0, 0, 1, 1, 0, 1]

In [4]:
# 1. 코퍼스 만들기
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1  # 패딩을 고려하여 +1 추가
print("단어 집합:", vocab_size)

단어 집합: 16


In [5]:
tokenizer.word_index

{'nice': 1,
 'great': 2,
 'best': 3,
 'amazing': 4,
 'stop': 5,
 'lies': 6,
 'pitiful': 7,
 'nerd': 8,
 'excellent': 9,
 'work': 10,
 'supreme': 11,
 'quality': 12,
 'bad': 13,
 'highly': 14,
 'respectable': 15}

In [6]:
# 2. 각 문장에 대해 정수인코딩 수행
X_encoded = tokenizer.texts_to_sequences(sentences)
print('정수 인코딩한 결과:\n', X_encoded)

정수 인코딩한 결과:
 [[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


In [8]:
## 3. 최대 길이로 모든 샘플에 대해 패딩 진행
max_len = max(len(encoded) for encoded in X_encoded) #가장 문장길이(sequence length)가 긴 것 구하기
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(y_train)
X_train

array([[ 1,  2,  3,  4],
       [ 5,  6,  0,  0],
       [ 7,  8,  0,  0],
       [ 9, 10,  0,  0],
       [11, 12,  0,  0],
       [13,  0,  0,  0],
       [14, 15,  0,  0]])

In [10]:
### 4. 모델 설계 : 이진분류 모델. 출력층에 1개의 뉴런을 배치하고 활성화함수는 시그모이드함수. 
## 손실함수는 binary_crossentropy, epoch = 100
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

In [12]:
embedding_dim = 4  # 하이퍼모수 행렬의 column 결정

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length = max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 0.6935 - accuracy: 0.4286 - 561ms/epoch - 561ms/step
Epoch 2/100
1/1 - 0s - loss: 0.6925 - accuracy: 0.4286 - 3ms/epoch - 3ms/step
Epoch 3/100
1/1 - 0s - loss: 0.6914 - accuracy: 0.4286 - 3ms/epoch - 3ms/step
Epoch 4/100
1/1 - 0s - loss: 0.6903 - accuracy: 0.4286 - 5ms/epoch - 5ms/step
Epoch 5/100
1/1 - 0s - loss: 0.6893 - accuracy: 0.4286 - 4ms/epoch - 4ms/step
Epoch 6/100
1/1 - 0s - loss: 0.6882 - accuracy: 0.4286 - 3ms/epoch - 3ms/step
Epoch 7/100
1/1 - 0s - loss: 0.6871 - accuracy: 0.4286 - 2ms/epoch - 2ms/step
Epoch 8/100
1/1 - 0s - loss: 0.6861 - accuracy: 0.5714 - 5ms/epoch - 5ms/step
Epoch 9/100
1/1 - 0s - loss: 0.6850 - accuracy: 0.5714 - 3ms/epoch - 3ms/step
Epoch 10/100
1/1 - 0s - loss: 0.6839 - accuracy: 0.7143 - 2ms/epoch - 2ms/step
Epoch 11/100
1/1 - 0s - loss: 0.6829 - accuracy: 0.7143 - 4ms/epoch - 4ms/step
Epoch 12/100
1/1 - 0s - loss: 0.6818 - accuracy: 0.7143 - 2ms/epoch - 2ms/step
Epoch 13/100
1/1 - 0s - loss: 0.6807 - accuracy: 1.0000 -

* 학습과정에서 현재 단어들의 임베딩 값은 출력층의 가중치와 함께 학습됨


### 2. 사전 훈련된 워드 임베딩(Pre-trained Word Embedding) 사용하기

* 사전 훈련된 GloVe와 Word2Vec 임베딩을 사용해서 모델을 훈련시키는 실습을 진행
    * GloVe 다운로드 링크 : http://nlp.stanford.edu/data/glove.6B.zip
    * Word2Vec 다운로드 링크 : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM

In [13]:
X_train

array([[ 1,  2,  3,  4],
       [ 5,  6,  0,  0],
       [ 7,  8,  0,  0],
       [ 9, 10,  0,  0],
       [11, 12,  0,  0],
       [13,  0,  0,  0],
       [14, 15,  0,  0]])

In [14]:
y_train

array([1, 0, 0, 1, 1, 0, 1])

In [15]:
##### 1) 사전 훈련된 GloVe 사용하기
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

In [16]:
urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename="glove.6B.zip")

('glove.6B.zip', <http.client.HTTPMessage at 0x19f8072c070>)

In [18]:
zf = zipfile.ZipFile('glove.6B.zip')
zf.extractall()
zf.close()

* glove.6B.100d.txt에 있는 모든 임베딩 벡터들을 불러오기. 파이썬의 자료구조 딕셔너리(dictionary)를 사용하며, 로드한 임베딩 벡터의 개수를 확인

In [26]:
embedding_dict = dict()

f = open('glove.6B.100d.txt', encoding='utf8')

i = 0
for line in f:
    word_vector = line.split()
    word = word_vector[0]
    
    # 100개 값을 가지는 array로 변환
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32')
    embedding_dict[word] = word_vector_arr
    
f.close()

In [27]:
print("%s 개의 Embedding vector가 있음" % len(embedding_dict))

400000 개의 Embedding vector가 있음


In [28]:
### 임의의 단어 'respectable'의 임베딩 벡터값과 크기를 출력
print(embedding_dict['respectable'])

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

In [29]:
print('벡터 차원수 : ', len(embedding_dict['respectable']))

벡터 차원수 :  100


In [30]:
embedding_matrix = np.zeros((vocab_size, 100))
embedding_matrix.shape

(16, 100)

In [31]:
## 기존 데이터의 각 단어와 맵핑된 정수값 확인
print(tokenizer.word_index)

{'nice': 1, 'great': 2, 'best': 3, 'amazing': 4, 'stop': 5, 'lies': 6, 'pitiful': 7, 'nerd': 8, 'excellent': 9, 'work': 10, 'supreme': 11, 'quality': 12, 'bad': 13, 'highly': 14, 'respectable': 15}


* 단어 'great'는 2인데, 사전 훈련된 GloVe에서는...

In [32]:
print(embedding_dict['great'])

[-0.013786   0.38216    0.53236    0.15261   -0.29694   -0.20558
 -0.41846   -0.58437   -0.77355   -0.87866   -0.37858   -0.18516
 -0.128     -0.20584   -0.22925   -0.42599    0.3725     0.26077
 -1.0702     0.62916   -0.091469   0.70348   -0.4973    -0.77691
  0.66045    0.09465   -0.44893    0.018917   0.33146   -0.35022
 -0.35789    0.030313   0.22253   -0.23236   -0.19719   -0.0053125
 -0.25848    0.58081   -0.10705   -0.17845   -0.16206    0.087086
  0.63029   -0.76649    0.51619    0.14073    1.019     -0.43136
  0.46138   -0.43585   -0.47568    0.19226    0.36065    0.78987
  0.088945  -2.7814    -0.15366    0.01015    1.1798     0.15168
 -0.050112   1.2626    -0.77527    0.36031    0.95761   -0.11385
  0.28035   -0.02591    0.31246   -0.15424    0.3778    -0.13599
  0.2946    -0.31579    0.42943    0.086969   0.019169  -0.27242
 -0.31696    0.37327    0.61997    0.13889    0.17188    0.30363
 -1.2776     0.044423  -0.52736   -0.88536   -0.19428   -0.61947
 -0.10146   -0.26301  

* 단어 집합의 모든 단어에 대해서 사전 훈련된 GloVe의 임베딩 벡터들을 맵핑한 후 'great'의 벡터값이 의도한 인덱스의 위치에 삽입되었는지 확인

In [33]:
for word, index in tokenizer.word_index.items():
    vector_value = embedding_dict.get(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

In [35]:
embedding_matrix[2]  # great

array([-0.013786  ,  0.38216001,  0.53236002,  0.15261   , -0.29694   ,
       -0.20558   , -0.41846001, -0.58437002, -0.77354997, -0.87866002,
       -0.37858   , -0.18516   , -0.12800001, -0.20584001, -0.22925   ,
       -0.42598999,  0.3725    ,  0.26076999, -1.07019997,  0.62915999,
       -0.091469  ,  0.70348001, -0.4973    , -0.77691001,  0.66044998,
        0.09465   , -0.44893   ,  0.018917  ,  0.33146   , -0.35021999,
       -0.35789001,  0.030313  ,  0.22253001, -0.23236001, -0.19719   ,
       -0.0053125 , -0.25848001,  0.58081001, -0.10705   , -0.17845   ,
       -0.16205999,  0.087086  ,  0.63028997, -0.76648998,  0.51618999,
        0.14072999,  1.01900005, -0.43136001,  0.46138   , -0.43584999,
       -0.47567999,  0.19226   ,  0.36065   ,  0.78987002,  0.088945  ,
       -2.78139997, -0.15366   ,  0.01015   ,  1.17980003,  0.15167999,
       -0.050112  ,  1.26259995, -0.77526999,  0.36030999,  0.95761001,
       -0.11385   ,  0.28035   , -0.02591   ,  0.31246001, -0.15

* 이제 Embedding layer에 embedding matrix 초기값 설정
* 사전 훈련된 워드 임베딩에서 100차원 값을 사용하고 있으므로 embedding layer의 output_dim의 인자값으로 100을 주어야 함
* 그리고 사전 훈련된 워드임베딩을 그대로 사용할 경우, 추가 훈련하지 않는다는 의미에서 trainable의 인자값을 False 선택

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

In [37]:
output_dim = 100

model = Sequential()
e = Embedding(vocab_size, output_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.7469 - acc: 0.4286 - 349ms/epoch - 349ms/step
Epoch 2/100
1/1 - 0s - loss: 0.7274 - acc: 0.4286 - 3ms/epoch - 3ms/step
Epoch 3/100
1/1 - 0s - loss: 0.7085 - acc: 0.4286 - 3ms/epoch - 3ms/step
Epoch 4/100
1/1 - 0s - loss: 0.6903 - acc: 0.4286 - 3ms/epoch - 3ms/step
Epoch 5/100
1/1 - 0s - loss: 0.6727 - acc: 0.4286 - 2ms/epoch - 2ms/step
Epoch 6/100
1/1 - 0s - loss: 0.6557 - acc: 0.4286 - 4ms/epoch - 4ms/step
Epoch 7/100
1/1 - 0s - loss: 0.6394 - acc: 0.5714 - 3ms/epoch - 3ms/step
Epoch 8/100
1/1 - 0s - loss: 0.6236 - acc: 0.5714 - 5ms/epoch - 5ms/step
Epoch 9/100
1/1 - 0s - loss: 0.6084 - acc: 0.5714 - 3ms/epoch - 3ms/step
Epoch 10/100
1/1 - 0s - loss: 0.5938 - acc: 0.5714 - 3ms/epoch - 3ms/step
Epoch 11/100
1/1 - 0s - loss: 0.5796 - acc: 0.5714 - 3ms/epoch - 3ms/step
Epoch 12/100
1/1 - 0s - loss: 0.5658 - acc: 0.5714 - 3ms/epoch - 3ms/step
Epoch 13/100
1/1 - 0s - loss: 0.5525 - acc: 0.5714 - 6ms/epoch - 6ms/step
Epoch 14/100
1/1 - 0s - loss: 0.5397 - acc:

* 정확도가 상당히 높아짐


#### 2) 사전 훈련된 Word2Vec 사용하기
* 구글 사전 훈련된 Word2Vec 모델 로드하여 사용

In [39]:
import gensim

urlretrieve("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", \
                           filename="GoogleNews-vectors-negative300.bin.gz")

('GoogleNews-vectors-negative300.bin.gz',
 <http.client.HTTPMessage at 0x19f942d4250>)

In [40]:
word2Vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [41]:
print('모델의 크기:', word2Vec_model.vectors.shape)

모델의 크기: (3000000, 300)


* 300 차원을 가진 벡터가 총 300만개 단어가 있음
* 모든 값이 0으로 채워진 임베딩 행렬을 만들기
    * 풀고자 하는 단어 집합의크기와 행과 300개 열을 가지는 행렬 생성

In [42]:
embedding_matrix = np.zeros((vocab_size, 300))
embedding_matrix.shape

(16, 300)

* word2Vec_model에서 특정단어를 입력하면 해당 단어의 임베딩벡터를 리턴받는데, 만약 word2vec_model에 특정단어의 임베딩 벡터가 없다면 None을 리턴하도록 함수 구현

In [43]:
def get_vector(word):
    if word in word2Vec_model:
        return word2Vec_model[word]
    else:
        return None

In [44]:
## 단어 집합(코퍼스)로부터 단어를 하나씩 호출하여 word2vec model 임베딩 벡터값이 존재하는지 확인
## 존재한다면 임베딩 행렬에 해당 단어의 인텍스 위치에 임베딩 벡터값 저장

In [48]:
i = 0
for word, index in tokenizer.word_index.items():
    vector_value = get_vector(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

* 기존 Word2vec_model에 저장되었던 단어 'nice' embedding vector 값 확인

In [49]:
print(word2Vec_model['nice'])

[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
 -0.01806641 -0.25390625  0.13183594  0.0859375   0.16113281  0.11083984
 -0.11083984 -0.0859375   0.0267334   0.34570312  0.15136719 -0.00415039
  0.10498047  0.04907227 -0.06982422  0.08642578  0.03198242 -0.02844238
 -0.15722656  0.11865234  0.36132812  0.00173187  0.05297852 -0.234375
  0.11767578  0.08642578 -0.01123047  0.25976562  0.28515625 -0.11669922
  0.38476562  0.07275391  0.01147461  0.03466797  0.18164062 -0.03955078
  0.04199219  0.01013184 -0.06054688  0.09765625  0.06689453  0.14648438
 -0.12011719  0.08447266 -0.06152344  0.06347656  0.3046875  -0.35546875
 -0.2890625   0.19628906 -0.33203125 -0.07128906  0.12792969  0.09619141
 -0.12158203 -0.08691406 -0.12890625  0.27734375  0.265625    0.1796875
  0.12695312  0.06298828 -0.34375    -0.05908203  0.0456543   0.171875
  0.08935547  0.14648438 -0.04638672 -0.00842285 -0.0279

In [50]:
tokenizer.word_index['nice']

1

In [51]:
embedding_matrix[1]

array([ 0.15820312,  0.10595703, -0.18945312,  0.38671875,  0.08349609,
       -0.26757812,  0.08349609,  0.11328125, -0.10400391,  0.17871094,
       -0.12353516, -0.22265625, -0.01806641, -0.25390625,  0.13183594,
        0.0859375 ,  0.16113281,  0.11083984, -0.11083984, -0.0859375 ,
        0.0267334 ,  0.34570312,  0.15136719, -0.00415039,  0.10498047,
        0.04907227, -0.06982422,  0.08642578,  0.03198242, -0.02844238,
       -0.15722656,  0.11865234,  0.36132812,  0.00173187,  0.05297852,
       -0.234375  ,  0.11767578,  0.08642578, -0.01123047,  0.25976562,
        0.28515625, -0.11669922,  0.38476562,  0.07275391,  0.01147461,
        0.03466797,  0.18164062, -0.03955078,  0.04199219,  0.01013184,
       -0.06054688,  0.09765625,  0.06689453,  0.14648438, -0.12011719,
        0.08447266, -0.06152344,  0.06347656,  0.3046875 , -0.35546875,
       -0.2890625 ,  0.19628906, -0.33203125, -0.07128906,  0.12792969,
        0.09619141, -0.12158203, -0.08691406, -0.12890625,  0.27

* 이제 Embedding에 사전 훈련된 embedding_matrix를 입력으로 넣어주고 모델을 학습

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input


In [53]:
model = Sequential()
model.add(Input(shape=(max_len,), dtype='int32'))
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 0.6738 - acc: 0.7143 - 659ms/epoch - 659ms/step
Epoch 2/100
1/1 - 0s - loss: 0.6556 - acc: 0.8571 - 4ms/epoch - 4ms/step
Epoch 3/100
1/1 - 0s - loss: 0.6379 - acc: 0.8571 - 4ms/epoch - 4ms/step
Epoch 4/100
1/1 - 0s - loss: 0.6207 - acc: 1.0000 - 3ms/epoch - 3ms/step
Epoch 5/100
1/1 - 0s - loss: 0.6041 - acc: 1.0000 - 3ms/epoch - 3ms/step
Epoch 6/100
1/1 - 0s - loss: 0.5880 - acc: 1.0000 - 2ms/epoch - 2ms/step
Epoch 7/100
1/1 - 0s - loss: 0.5725 - acc: 1.0000 - 3ms/epoch - 3ms/step
Epoch 8/100
1/1 - 0s - loss: 0.5575 - acc: 1.0000 - 3ms/epoch - 3ms/step
Epoch 9/100
1/1 - 0s - loss: 0.5430 - acc: 1.0000 - 3ms/epoch - 3ms/step
Epoch 10/100
1/1 - 0s - loss: 0.5290 - acc: 1.0000 - 4ms/epoch - 4ms/step
Epoch 11/100
1/1 - 0s - loss: 0.5155 - acc: 1.0000 - 5ms/epoch - 5ms/step
Epoch 12/100
1/1 - 0s - loss: 0.5025 - acc: 1.0000 - 4ms/epoch - 4ms/step
Epoch 13/100
1/1 - 0s - loss: 0.4899 - acc: 1.0000 - 4ms/epoch - 4ms/step
Epoch 14/100
1/1 - 0s - loss: 0.4777 - acc: